In [1]:
# Libraries
import itertools
from importlib import resources
import pandas as pd
import numpy as np
from tqdm import tqdm

from drdt.helper_functions import DecisionRuleCreatorFromDecisionTable, Reduction, R_SR, R_AD, SAlphaStep, SPlus, SMax, NCover, NGreedy
from drdt.algorithms import DynamicProgrammingAlgorithms, A_C_N, A_C_G


# Loading Data

In [2]:
with resources.path('datasets.DecisionRuleSystems', 'DRS_car_evaluation') as dataset_path:
    S = pd.read_csv(dataset_path).sample(n=10, random_state=42).applymap(lambda x: str(x) if pd.notnull(x) else x)
S = S.dropna(axis=1, how='all') # Drop the columns with all None
S

,buying,maint,doors,persons,lug_boot,safety,class
599,NaN,NaN,NaN,2,NaN,NaN,unacc
1201,med,NaN,2,4,med,med,acc
628,NaN,NaN,NaN,2,NaN,NaN,unacc
1498,NaN,high,5more,NaN,NaN,med,acc
1263,NaN,NaN,NaN,NaN,NaN,low,unacc
931,med,vhigh,4,4,med,NaN,acc
23,vhigh,vhigh,NaN,NaN,NaN,NaN,unacc
844,NaN,NaN,NaN,2,NaN,NaN,unacc
964,NaN,vhigh,NaN,NaN,small,med,unacc
764,NaN,NaN,NaN,2,NaN,NaN,unacc


# Dataset Analyses

In [3]:
print(f"Number of features  = {len(S.columns)-1}")

Number of features  = 6


In [4]:
print(f"Does # of features = d  = {not S.dropna().empty}")

Does # of features = d  = False


In [5]:
# Count non-NaN values for each row
non_nan_counts = S.count(axis=1)

# Find the index
max_non_nan_row_index = non_nan_counts.idxmax()

# Retrieve the row
max_non_nan_row = S.loc[max_non_nan_row_index]

# Number of non-NaN values in the row
max_non_nan_count = non_nan_counts[max_non_nan_row_index]

print(f"d of  = {max_non_nan_count - 1}") # remove 1 because of last class column

d of  = 5


In [6]:
P = S
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for AR = {len(B)}")

Length of Node Cover for AR = 5


In [7]:
P = R_SR(S)
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for SR = {len(B)}")

Length of Node Cover for SR = 5


In [8]:
P = R_AD(S)
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for AD = {len(B)}")

Length of Node Cover for AD = 5


In [9]:
P = S
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for AR = {len(B)}")

Length of Node Cover greedy for AR = 1


In [10]:
P = R_SR(S)
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for SR = {len(B)}")

Length of Node Cover greedy for SR = 1


In [11]:
P = R_AD(S)
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for AD = {len(B)}")

Length of Node Cover greedy for AD = 1


# Dynamic Programming

In [12]:
alg = DynamicProgrammingAlgorithms(C="AR")
depth = alg.A_DP(S)
print(f"DP AR = {depth}")

DP AR = 6


In [13]:
alg = DynamicProgrammingAlgorithms(C="EAR")
depth = alg.A_DP(S)
print(f"DP EAR = {depth}")

DP EAR = 6


In [14]:
alg = DynamicProgrammingAlgorithms(C="SR")
depth = alg.A_DP(S)
print(f"DP SR = {depth}")

DP SR = 6


In [15]:
alg = DynamicProgrammingAlgorithms(C="ESR")
depth = alg.A_DP(S)
print(f"DP ESR = {depth}")

DP ESR = 6


In [16]:
alg = DynamicProgrammingAlgorithms(C="AD")
depth = alg.A_DP(S)
print(f"DP AD = {depth}")

DP AD = 6


In [17]:
alg = DynamicProgrammingAlgorithms(C="EAD")
depth = alg.A_DP(S)
print(f"DP EAD = {depth}")

DP EAD = 6


# Combinations

In [18]:
column_values = []

for column in S.columns[:-1]: #ignoring class column
    column_values.append(list(S[column].dropna().unique()))
    
# All possible combinations    
combinations = list(itertools.product(*column_values))


In [19]:
column_values_extension = []

for column in S.columns[:-1]: #ignoring class column
    column_values_extension.append(list(S[column].dropna().unique()) + ['*'])
    
# All possible combinations with extension    
extended_combinations = list(itertools.product(*column_values_extension))


# Node Cover

In [20]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NC AR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|██████████████████████████████████████████| 96/96 [00:00<00:00, 178.94it/s]

NC AR
(Max Depth, Min Depth, Average Depth) = (6, 5, 5.708333333333333)


In [21]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NC EAR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|████████████████████████████████████████| 972/972 [00:04<00:00, 202.35it/s]

NC EAR
(Max Depth, Min Depth, Average Depth) = (6, 5, 5.453703703703703)


In [22]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NC SR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|██████████████████████████████████████████| 96/96 [00:00<00:00, 118.26it/s]

NC SR
(Max Depth, Min Depth, Average Depth) = (6, 5, 5.229166666666667)


In [23]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NC ESR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|████████████████████████████████████████| 972/972 [00:07<00:00, 123.75it/s]

NC ESR
(Max Depth, Min Depth, Average Depth) = (6, 5, 5.228395061728395)


In [24]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NC AD")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|██████████████████████████████████████████| 96/96 [00:00<00:00, 136.20it/s]

NC AD
(Max Depth, Min Depth, Average Depth) = (6, 5, 5.333333333333333)


In [25]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NC EAD")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|████████████████████████████████████████| 972/972 [00:06<00:00, 146.21it/s]

NC EAD
(Max Depth, Min Depth, Average Depth) = (6, 5, 5.2592592592592595)


# Node Cover Greedy

In [26]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NCgreedy AR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|███████████████████████████████████████████| 96/96 [00:01<00:00, 73.31it/s]

NCgreedy AR
(Max Depth, Min Depth, Average Depth) = (6, 5, 5.25)


In [27]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NCgreedy EAR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|█████████████████████████████████████████| 972/972 [00:11<00:00, 82.64it/s]

NCgreedy EAR
(Max Depth, Min Depth, Average Depth) = (6, 4, 4.888888888888889)


In [28]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NCgreedy SR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|███████████████████████████████████████████| 96/96 [00:01<00:00, 55.23it/s]

NCgreedy SR
(Max Depth, Min Depth, Average Depth) = (6, 2, 4.458333333333333)


In [29]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NCgreedy ESR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|█████████████████████████████████████████| 972/972 [00:16<00:00, 58.26it/s]

NCgreedy ESR
(Max Depth, Min Depth, Average Depth) = (6, 2, 4.546296296296297)


In [30]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NCgreedy AD")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|███████████████████████████████████████████| 96/96 [00:01<00:00, 64.68it/s]

NCgreedy AD
(Max Depth, Min Depth, Average Depth) = (6, 2, 4.458333333333333)


In [31]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NCgreedy EAD")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|█████████████████████████████████████████| 972/972 [00:14<00:00, 67.71it/s]

NCgreedy EAD
(Max Depth, Min Depth, Average Depth) = (6, 2, 4.546296296296297)


# Greedy

In [32]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("Greedy AR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|██████████████████████████████████████████| 96/96 [00:00<00:00, 201.11it/s]

Greedy AR
(Max Depth, Min Depth, Average Depth) = (6, 3, 4.1875)


In [33]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("Greedy EAR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|████████████████████████████████████████| 972/972 [00:04<00:00, 241.67it/s]

Greedy EAR
(Max Depth, Min Depth, Average Depth) = (6, 3, 3.6574074074074074)


In [34]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="SR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("Greedy SR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|██████████████████████████████████████████| 96/96 [00:00<00:00, 105.32it/s]

Greedy SR
(Max Depth, Min Depth, Average Depth) = (6, 2, 3.0208333333333335)


In [35]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="ESR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("Greedy ESR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|████████████████████████████████████████| 972/972 [00:08<00:00, 111.06it/s]

Greedy ESR
(Max Depth, Min Depth, Average Depth) = (6, 2, 3.0462962962962963)


In [36]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("Greedy AD")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|██████████████████████████████████████████| 96/96 [00:00<00:00, 130.98it/s]

Greedy AD
(Max Depth, Min Depth, Average Depth) = (6, 2, 3.125)


In [37]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("Greedy EAD")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|████████████████████████████████████████| 972/972 [00:06<00:00, 142.58it/s]

Greedy EAD
(Max Depth, Min Depth, Average Depth) = (6, 2, 3.0771604938271606)
